In [ ]:
%pip install steamship===2.2.0b2

# Demo: Audio Markdown Package
## Transcribe audio using Steamship's Audio Markdown Package 


This notebook demonstrates how to create and use the `audio-markdown` package. 

Instances of the package will be created in such a way that each instance will have its own workspace where data is stored. Workspaces can be used to create personal data vaults for clients as they are isolated from each other.

In [ ]:
from steamship import Steamship
from steamship.base import TaskState

In [ ]:
import time

### Constants

Using this notebook requires a Steamship API key. If you do not have one, you can create one by typing the following in a terminal. If you opened this notebook using `npx try-steamship audio-markdown` you will already have an API key configured in `~/steamship.json`. 

> npm install -g @steamship/cli && steamship login

# Create a new instance of the package

In [ ]:
instance = Steamship.use("audio-markdown-staging", "audio-markdown-client-test-001")

In [ ]:
print(
    f"""
{'Invocation URL': <20}: {instance.invocation_url}
{'Instance ID': <20}: {instance.id}
{'Version ID': <20}: {instance.package_version_id} 
{'Package ID': <20}: {instance.package_id}
{'Workspace ID': <20}: {instance.workspace_id}
"""
)

# Submitting audio files for Markdown generation

The `audio-markdown` package transcribes audio files that are accessible via URL. Any publically accessible URL will work including pre-signed url's to S3 or Google Storage.

In the code sample below we show you how to submit an URL of your audio file to the `transcribe_url` API.

The `POST` request triggers the asynchronous transcription and markdown generation from your file. 

After submitting your `POST` request you will receive a response that includes a `task_id` and `status` key. 

The `status` key shows you the status of your analysis task. It will start with `"waiting"`, and then proceed to `"processing"`, and finally to `"succeeded"` or `"failed"`. 

In [ ]:
# URL pointing to an audio file
# Replace this with your own audio file
audio_url = "https://sndup.net/wdp4/d"

In [ ]:
# Transcribe and analyze your audio file
transcribe_response = instance.invoke("transcribe_url", url=audio_url)

task_id = transcribe_response["task_id"]
status = transcribe_response["status"]

print(
    f"""
task ID: {task_id}
status: {status}
"""
)

### Retrieving Markdown results

As your file is being processed the `"status"` will go from `"waiting"` to `"processing"` to `"succeeded"` or `"failed"`. You can check in on the progress of your task using the `"task_id"` by calling the `get_markdown` endpoint. 

You'll have to make repeated requests untill the status converges to `"succeeded"` or `"failed"`. Once the `status` key is set to `"succeeded"`, you'll see a `markdown` key that represents the generated Markdown. 

In [ ]:
retries = 0
while retries <= 100 and status != TaskState.succeeded:
    response = instance.invoke("get_markdown", task_id=task_id)
    status = response['status']
    if status == TaskState.failed:
        print(f"[FAILED] {response.status_message}")
        break

    print(f"[Try {retries}] Transcription {status}.")
    if status == TaskState.succeeded:
        break
    time.sleep(2)
    retries += 1

markdown_file = response["markdown"]


In [ ]:
from IPython.display import display, Markdown
display(Markdown(markdown_file))